<a href="https://colab.research.google.com/github/Naveenand/Rag/blob/main/Rag_bot_Q%26A.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings
warnings.filterwarnings("ignore")

!pip install -qU \
    langchain \
    openai \
    pinecone-client \
    tiktoken  \
    langchain_community \
    langchain_openai \
    pypdf

In [ ]:
!pip install langchain_community pypdf

In [ ]:
from textwrap import wrap
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.document_loaders import PyPDFLoader

In [ ]:
loader = PyPDFLoader("/content/apple_annual_report_2023.pdf")
pages = loader.load_and_split()

In [ ]:
pages[5]

Document(page_content='The Company is focused on expanding its market opportunities related to smartphones, personal computers, tablets, wearables \nand accessories, and services. The Company faces substantial competition in these markets from companies that have \nsignificant technical, marketing, distribution and other resources, as well as established hardware, software, and service offerings \nwith large customer bases. In addition, some of the Company’s competitors have broader product lines, lower-priced products \nand a larger installed base of active devices. Competition has been particularly intense as competitors have aggressively cut \nprices and lowered product margins. Certain competitors have the resources, experience or cost structures to provide products \nat little or no profit or even at a loss. The Company’s services compete with business models that provide content to users for \nfree and use illegitimate means to obtain third-party digital content and applications.

In [ ]:
# Filter the empty strings
pdf_texts = [text for text in pages if text]

print(pdf_texts[0])
print(len(pdf_texts))

page_content='UNITED STATES\nSECURITIES AND EXCHANGE COMMISSION\nWashington, D.C. 20549\nFORM 10-K\n(Mark One)\n☒    ANNUAL REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the fiscal year ended September\xa024, 2022\nor\n☐    TRANSITION REPORT PURSUANT TO SECTION 13 OR 15(d) OF THE SECURITIES EXCHANGE ACT OF 1934\nFor the transition period from \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0  to \xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0\xa0 .\nCommission File Number: 001-36743\nApple Inc.\n(Exact name of Registrant as specified in its charter)\nCalifornia 94-2404110\n(State or other jurisdiction\nof incorporation or organization)(I.R.S. Employer Identification No.)\nOne Apple Park Way\nCupertino , California 95014\n(Address of principal executive offices) (Zip Code)\n(408) 996-1010\n(Registrant’s telephone number, including area code)\nSecurities registered pursuant to Section 12(b) of the Act:\nTitle of each classTrading \nsymbol(s) Name of each 

In [ ]:
# Set up the RecursiveCharacterTextSplitter, then Split the documents
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=0)
texts = text_splitter.split_documents(pdf_texts)

In [ ]:
print(texts[5])
print(len(texts))
print(type(texts))

page_content='III of this Annual Report on Form 10-K where indicated. The Registrant’s definitive proxy statement will be filed with the U.S. Securities and \nExchange Commission within 120 days after the end of the fiscal year to which this report relates.' metadata={'source': '/content/apple_annual_report_2023.pdf', 'page': 1}
372
<class 'list'>


#Pinecone and OpenAI Embedding setup

In [ ]:
from google.colab import userdata
import os
import pinecone

In [ ]:
import pinecone

pinecone.init(api_key='YOUR_API_KEY', environment='us-east1-gcp')

## The following example creates an index without a metadata
## configuration. By default, Pinecone indexes all metadata.
#'rag-project is a index name, so give your own index name
index = pinecone.create_index('rag-project', dimension=1536)

In [ ]:
index = pinecone.Index('rag-project')

In [ ]:
# Examine pinecone index. Delete all vectors, if you want to start fresh
index.describe_index_stats()
#index.delete(deleteAll='true', namespace='rag-project')

{'dimension': 1536,
 'index_fullness': 0.0,
 'namespaces': {},
 'total_vector_count': 0}

#Openai


In [ ]:
import os
import openai
from openai import OpenAI
import getpass
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_community.vectorstores import Pinecone
os.environ['OPENAI_API_KEY'] = getpass.getpass('OpenAI API Key: OPENAI_API_KEY')

In [ ]:
openai_client = OpenAI()
#assigning the model
model = "gpt-3.5-turbo"

# Prepare the embedding so that we can pass it to the pinecone call in the next step
embeddings = OpenAIEmbeddings(model="text-embedding-ada-002")

In [ ]:
index_name = 'rag-project'

In [ ]:
# The OpenAI embedding model `text-embedding-ada-002 uses 1536 dimensions`
docsearch = Pinecone.from_documents(texts,embeddings,index_name=index_name)
#docsearch = Pinecone.from_existing_index(index_name, embeddings)

In [ ]:
query = "What has been the investment in research and development?"
docs = docsearch.similarity_search(query, k=5)

In [ ]:
docs

[Document(page_content='The Company’s ability to compete successfully depends heavily on ensuring the continuing and timely introduction of innovative \nnew products, services and technologies to the marketplace. The Company designs and develops nearly the entire solution for \nits products, including the hardware, operating system, numerous software applications and related services. As a result, the \nCompany must make significant investments in R&D. There can be no assurance these investments will achieve expected \nreturns, and the Company may not be able to develop and market new products and services successfully.\nThe Company currently holds a significant number of patents, trademarks and copyrights and has registered, and applied to \nregister, additional patents, trademarks and copyrights. In contrast, many of the Company’s competitors seek to compete \nprimarily through aggressive pricing and very low cost structures, and by imitating the Company’s products and infringing on'

In [ ]:
def augment_query_generated(query, model):
    messages = [
        {
            "role": "system",
            "content": "You are a helpful expert financial research assistant. Provide an example answer to the given question, that might be found in a document like an annual report. "
        },
        {"role": "user", "content": query}
    ]

    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content

In [ ]:
original_query = "Was there significant turnover in the executive team?"
hypothetical_answer = augment_query_generated(original_query,model)

joint_query = f"{original_query} {hypothetical_answer}"
for element in wrap(joint_query):
  print(element)

Was there significant turnover in the executive team? Yes, there was
significant turnover in the executive team during the year. In the
annual report, it is mentioned that three key executives resigned and
were replaced by new individuals. The Chief Financial Officer, Chief
Operating Officer, and Chief Marketing Officer all stepped down from
their positions and were succeeded by new hires with extensive
experience in their respective fields. The report highlights that the
company underwent a deliberate effort to refresh its leadership team
and bring in new perspectives and expertise to drive the company's
growth strategy. The turnover in the executive team was seen as a
positive development, further reinforcing the company's commitment to
adapt to changing market dynamics and enhance its competitive
position.


In [ ]:
results = docsearch.similarity_search(query, k=3)
retrieved_documents = results[0:4]

for doc in retrieved_documents:
    print(doc.page_content)
    print('')

The Company’s ability to compete successfully depends heavily on ensuring the continuing and timely introduction of innovative 
new products, services and technologies to the marketplace. The Company designs and develops nearly the entire solution for 
its products, including the hardware, operating system, numerous software applications and related services. As a result, the 
Company must make significant investments in R&D. There can be no assurance these investments will achieve expected 
returns, and the Company may not be able to develop and market new products and services successfully.
The Company currently holds a significant number of patents, trademarks and copyrights and has registered, and applied to 
register, additional patents, trademarks and copyrights. In contrast, many of the Company’s competitors seek to compete 
primarily through aggressive pricing and very low cost structures, and by imitating the Company’s products and infringing on

Asia, with some Mac computers 

#Genetating Multiple Query

In [ ]:
def augment_multiple_query(query, model):
    messages = [
        {
            "role": "system",
            "content": "You are a helpful expert financial research assistant. Your users are asking questions about an annual report. "
            "Suggest up to five additional related questions to help them find the information they need, for the provided question. "
            "Suggest only short questions without compound sentences. Suggest a variety of questions that cover different aspects of the topic."
            "Make sure they are complete questions, and that they are related to the original question."
            "Output one question per line. Do not number the questions."
        },
        {"role": "user", "content": query}
    ]

    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    content = content.split("\n")
    return content

In [ ]:
original_query = "What are the main sources of revenue for the company?"
augmented_queries = augment_multiple_query(original_query,model)

for query in augmented_queries:
    print(query)

What is the company's net income for the year? 
What were the company's operating expenses for the year? 
What is the breakdown of revenue by product/service category? 
What is the company's revenue growth compared to previous years? 
What percentage of the company's revenue comes from international markets?


In [ ]:
query = [original_query] + augmented_queries

In [ ]:
def perform_similarity_search(question, k= 10):
    results = docsearch.similarity_search(question, k=k)
    return {question: results}

In [ ]:
query_results_dict = {}
for qes in query:
    query_results_dict.update(perform_similarity_search(qes))

In [ ]:
print(query_results_dict)

{'What are the main sources of revenue for the company?': [Document(page_content='Business Seasonality and Product Introductions\nThe Company has historically experienced higher net sales in its first quarter compared to other quarters in its fiscal year due in \npart to seasonal holiday demand. Additionally, new product and service introductions can significantly impact net sales, cost of \nsales and operating expenses. The timing of product introductions can also impact the Company’s net sales to its indirect \ndistribution channels as these channels are filled with new inventory following a product launch, and channel inventory of an \nolder product often declines as the launch of a newer product approaches. Net sales can also be affected when consumers and \ndistributors anticipate a product introduction.\nHuman Capital\nThe Company believes it has a talented, motivated and dedicated team, and works to create an inclusive, safe and supportive \nenvironment for all of its team membe

In [ ]:
for document_list in query_results_dict.values():
    for document in document_list:
        # Process each document here
        print(document)
        print('-'*100)

page_content='Business Seasonality and Product Introductions\nThe Company has historically experienced higher net sales in its first quarter compared to other quarters in its fiscal year due in \npart to seasonal holiday demand. Additionally, new product and service introductions can significantly impact net sales, cost of \nsales and operating expenses. The timing of product introductions can also impact the Company’s net sales to its indirect \ndistribution channels as these channels are filled with new inventory following a product launch, and channel inventory of an \nolder product often declines as the launch of a newer product approaches. Net sales can also be affected when consumers and \ndistributors anticipate a product introduction.\nHuman Capital\nThe Company believes it has a talented, motivated and dedicated team, and works to create an inclusive, safe and supportive \nenvironment for all of its team members. As of September\xa0 24, 2022 , the Company had approximately 164

In [ ]:
# Deduplicate the retrieved documents
unique_documents = set()
for documents in query_results_dict.values():
    for document in documents:
        unique_documents.add(document.page_content)

unique_documents = list(unique_documents)

In [ ]:
unique_documents

['the sales price of the respective product.\n(2) Wearables, Home and Accessories net sales include sales of AirPods, Apple TV, Apple Watch, Beats products, HomePod \nmini and accessories.\n(3) Services net sales include sales from the Company’s advertising, AppleCare, cloud, digital content, payment and other \nservices. Services net sales also include amortization of the deferred value of services bundled in the sales price of certain \nproducts.\n(4) Includes $7.5 billion  of revenue recognized in 2022  that was included in deferred revenue as of September\xa025, 2021 , $6.7 \nbillion  of revenue recognized in 2021  that was included in deferred revenue as of September\xa026, 2020 , and $5.0 billion  of \nrevenue recognized in 2020  that was included in deferred revenue as of September\xa028, 2019 .\nThe Company’s proportion of net sales by disaggregated revenue source was generally consistent for each reportable segment',
 'Apple Inc.\nCONSO LIDATED STATEMENTS OF CASH FLOWS\n(In mi

#CrossEncoder ReRanking


In [ ]:
!pip install sentence_transformers

In [ ]:
from sentence_transformers import CrossEncoder
cross_encoder = CrossEncoder('cross-encoder/ms-marco-MiniLM-L-6-v2')

config.json:   0%|          | 0.00/794 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/90.9M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/316 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
original_query

'What are the main sources of revenue for the company?'

In [ ]:
pairs = []
for doc in unique_documents:
    pairs.append([original_query, doc])

In [ ]:
pairs[0]

['What are the main sources of revenue for the company?',
 'the sales price of the respective product.\n(2) Wearables, Home and Accessories net sales include sales of AirPods, Apple TV, Apple Watch, Beats products, HomePod \nmini and accessories.\n(3) Services net sales include sales from the Company’s advertising, AppleCare, cloud, digital content, payment and other \nservices. Services net sales also include amortization of the deferred value of services bundled in the sales price of certain \nproducts.\n(4) Includes $7.5 billion  of revenue recognized in 2022  that was included in deferred revenue as of September\xa025, 2021 , $6.7 \nbillion  of revenue recognized in 2021  that was included in deferred revenue as of September\xa026, 2020 , and $5.0 billion  of \nrevenue recognized in 2020  that was included in deferred revenue as of September\xa028, 2019 .\nThe Company’s proportion of net sales by disaggregated revenue source was generally consistent for each reportable segment']

In [ ]:
scores = cross_encoder.predict(pairs)

In [ ]:
print("Scores:")
for score in scores:
    print(score)

Scores:
-2.1661196
-9.4072275
-3.027835
-10.799191
-7.3105316
-7.3626065
-10.770123
-5.8069496
-5.967999
-8.474387
-3.8404212
-1.6677408
-4.403803
-10.176614
-10.318494
-8.996105
-8.645668
-10.185031
-9.6425
-10.580616
-10.181423
-10.445876
-10.200811
-7.6111503
-10.647938
-2.5745893
-9.271004
-5.888918
-9.44285
-7.6121073
-8.660635
-9.366646
-10.446329
-5.716876


In [ ]:
import numpy as np

In [ ]:
new_ordering = []

In [ ]:
print("New Ordering:")
for o in np.argsort(scores)[::-1]:
  new_ordering.append(o)
  print(o)

New Ordering:
11
0
25
2
10
12
33
7
27
8
4
5
23
29
9
16
30
15
26
31
1
28
18
13
20
17
22
14
21
32
19
24
6
3


In [ ]:
# increase the input documents if the model have larger context window or large tokens window
new_ordering = new_ordering[0:2]

In [ ]:
ordered_list = [pairs[i] for i in new_ordering]

In [ ]:
cleaned_list = [[sublist[1]] for sublist in ordered_list]

In [ ]:
cleaned_list

[['For the sale of third-party products where the Company obtains control of the product before transferring it to the customer, the \nCompany recognizes revenue based on the gross amount billed to customers. The Company considers multiple factors when \ndetermining whether it obtains control of third-party products, including evaluating if it can establish the price of the product, \nretains inventory risk for tangible products or has the responsibility for ensuring acceptability of the product. For third-party \napplications sold through the App Store and certain digital content sold through the Company’s other digital content stores, the \nCompany does not obtain control of the product before transferring it to the customer. Therefore, the Company accounts for such \nsales on a net basis by recognizing in Services net sales only the commission it retains.\nThe Company records revenue net of taxes collected from customers that are remitted to governmental authorities, with the'],
 ['

In [ ]:
# Convert each sublist to a string with newline characters
string_with_newline = '\n'.join([' '.join(sublist) for sublist in cleaned_list])

In [ ]:
string_with_newline

'For the sale of third-party products where the Company obtains control of the product before transferring it to the customer, the \nCompany recognizes revenue based on the gross amount billed to customers. The Company considers multiple factors when \ndetermining whether it obtains control of third-party products, including evaluating if it can establish the price of the product, \nretains inventory risk for tangible products or has the responsibility for ensuring acceptability of the product. For third-party \napplications sold through the App Store and certain digital content sold through the Company’s other digital content stores, the \nCompany does not obtain control of the product before transferring it to the customer. Therefore, the Company accounts for such \nsales on a net basis by recognizing in Services net sales only the commission it retains.\nThe Company records revenue net of taxes collected from customers that are remitted to governmental authorities, with the\nthe sal

In [ ]:
def rag(query, retrieved_documents,model):
    information = "\n\n".join(retrieved_documents)

    messages = [
        {
            "role": "system",
            "content": "You are a helpful expert financial research assistant. Your users are asking questions about information contained in an annual report."
            "You will be shown the user's question, and the relevant information from the annual report. Answer the user's question using only this information."
        },
        {"role": "user", "content": f"Question: {query}. \n Information: {information}"}
    ]

    response = openai_client.chat.completions.create(
        model=model,
        messages=messages,
    )
    content = response.choices[0].message.content
    return content

In [ ]:
rag(original_query,string_with_newline,model)